In [ ]:
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature

import xgboost as xgb
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [7]:
# Creating training and testing dataset
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Training data
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

# Test Data
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

# Validation data
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

data_signature = infer_signature(train_x, train_y)

In [8]:
# Normalizing target label for Sparse Categorical Cross Entropy

train_y -= 3
val_y -= 3
test_y -= 3

train_y = train_y.astype(np.float32)
val_y = val_y.astype(np.float32)
test_y = test_y.astype(np.float32)


In [9]:
# Hyperopt space | params
space = {
    "max_depth": hp.choice("max_depth", [3, 4, 5, 6, 7, 8]),
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "n_estimators": hp.choice("n_estimators", [50, 100, 200, 300])
}

In [10]:
def objective(params):
    with mlflow.start_run(nested=True):
        # Model setup
        model = xgb.XGBRegressor(
            objective='reg:squarederror',
            random_state=42,
            tree_method='hist',  # Faster on CPUs; remove for GPU
            **params
        )

        # Fit
        model.fit(
            train_x, train_y,
            eval_set=[(val_x, val_y)],
            # early_stopping_rounds=10,
            verbose=False
        )

        # Predict and evaluate
        y_pred = model.predict(test_x)
        rmse = np.sqrt(mean_squared_error(test_y, y_pred))
        mae = mean_absolute_error(test_y, y_pred)
        r2 = r2_score(test_y, y_pred)

        # Log params and metrics
        mlflow.log_params(params)
        mlflow.log_metric("test_rmse", rmse)
        mlflow.log_metric("test_mae", mae)
        mlflow.log_metric("test_r2", r2)

        # Log the model
        mlflow.xgboost.log_model(model, "model", signature=data_signature)

        return {
            "loss": rmse,
            "status": STATUS_OK,
            "model": model,
            "test_rmse": rmse,
            "test_mae": mae,
            "test_r2": r2
        }

In [11]:
mlflow.set_experiment("/wine-quality-xgboost")

2025/07/09 01:38:54 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality-xgboost' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/Krish%20Naik%20Youtube/MLOps%20Bootcamp/mlops-practicals/Section%208%20-%20ANN%20with%20MLFlow/mlruns/309604394835588156', creation_time=1752039534595, experiment_id='309604394835588156', last_update_time=1752039534595, lifecycle_stage='active', name='/wine-quality-xgboost', tags={}>

In [14]:
with mlflow.start_run(run_name="WineQuality_XGBoost_Hyperopt"):
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=20,  # You can increase for more thorough search
        trials=trials,
        rstate=np.random.default_rng(42)
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best metrics and model
    mlflow.log_params(best)
    mlflow.log_metric("RMSE", best_run["test_rmse"])
    mlflow.log_metric("MAE", best_run["test_mae"])
    mlflow.log_metric("R2", best_run["test_r2"])
    mlflow.xgboost.log_model(best_run["model"], "best_model", signature=data_signature)

    print(f"Best Parameters: {best}")
    print(f"Best RMSE: {best_run['test_rmse']}")
    print(f"Best MAE: {best_run['test_mae']}")
    print(f"Best R^2: {best_run['test_r2']}")

100%|██████████| 20/20 [01:01<00:00,  3.08s/trial, best loss: 0.6128917336463928]
Best Parameters: {'learning_rate': 0.1594042854587566, 'max_depth': 5, 'n_estimators': 2}
Best RMSE: 0.6128917336463928
Best MAE: 0.4168238937854767
Best R^2: 0.514978313957718
